In [7]:
import streamlit as st
from streamlit_shap import st_shap
import shap
from sklearn.model_selection import train_test_split
import xgboost
import numpy as np
import pandas as pd

st.set_page_config(layout="wide")

# shap lib로 데이터셋 로드
@st.experimental_memo
def load_data():
    return shap.datasets.adult()

@st.experimental_memo
def load_model(X, y):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=7)
    d_train = xgboost.DMatrix(X_train, label=y_train)
    d_test = xgboost.DMatrix(X_test, label=y_test)
    params = {
        "eta": 0.01,
        "objective": "binary:logistic",
        "subsample": 0.5,
        "base_score": np.mean(y_train),
        "eval_metric": "logloss",
        "n_jobs": -1,
    }
    model = xgboost.train(params, d_train, 10, evals = [(d_test, "test")], verbose_eval=100, early_stopping_rounds=20)
    return model

# 앱 제목 표시
st.title("`streamlit-shap`로 Streamlit 앱에서 SHAP 플롯 표시하기")


# 앱 설명 제공 정보 박스
with st.expander('앱에 대하여'):
    st.markdown('''[`streamlit-shap`](https://github.com/snehankekre/streamlit-shap)는 [SHAP](https://github.com/slundberg/shap) 플롯을 [Streamlit](https://streamlit.io/)에서 표시하기 위한 래퍼를 제공하는 Streamlit 컴포넌트입니다.
                    이 라이브러리는 저희 내부 직원인 [스네한 케크레](https://github.com/snehankekre)가 개발했으며, [Streamlit 문서화](https://docs.streamlit.io/) 웹사이트도 관리합니다.
                ''')

st.header('입력 데이터')
X, y = load_data()
X_display, y_display = shap.datasets.adult(display=True)

with st.expander('데이터에 대하여'):
    st.write('예시 데이터셋으로 성인 인구 조사 데이터를 사용합니다.')
with st.expander('X'):
    st.dataframe(X)
with st.expander('y'):
    st.dataframe(y)

st.header('SHAP 출력')

# XGBoost 모델 훈련
X, y = load_data()
X_display, y_display = shap.datasets.adult(display=True)

model = load_model(X, y)

# SHAP 값 계산
explainer = shap.Explainer(model, X)
shap_values = explainer(X)

with st.expander('워터폴 플롯'):
    st_shap(shap.plots.waterfall(shap_values[0]), height=300)
with st.expander('비스웜 플롯'):
    st_shap(shap.plots.beeswarm(shap_values), height=300)

explainer = shap.TreeExplainer(model)
shap_values = explainer.shap_values(X)

with st.expander('포스 플롯'):
    st.subheader('첫 번째 데이터 인스턴스')
    st_shap(shap.force_plot(explainer.expected_value, shap_values[0,:], X_display.iloc[0,:]), height=200, width=1000)
    st.subheader('첫 천 번째 데이터 인스턴스')
    st_shap(shap.force_plot(explainer.expected_value, shap_values[:1000,:], X_display.iloc[:1000,:]), height=400, width=1000)


2025-06-27 16:22:14.867 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


AttributeError: module 'streamlit' has no attribute 'experimental_memo'

In [4]:
pip install streamlit_shap

  Using cached streamlit_shap-1.0.2-py3-none-any.whl.metadata (2.7 kB)
  Using cached scipy-1.16.0-cp313-cp313-win_amd64.whl.metadata (60 kB)
  Using cached tqdm-4.67.1-py3-none-any.whl.metadata (57 kB)
  Using cached slicer-0.0.8-py3-none-any.whl.metadata (4.0 kB)
  Using cached cloudpickle-3.1.1-py3-none-any.whl.metadata (7.1 kB)
  Using cached joblib-1.5.1-py3-none-any.whl.metadata (5.6 kB)
  Using cached threadpoolctl-3.6.0-py3-none-any.whl.metadata (13 kB)
Using cached streamlit_shap-1.0.2-py3-none-any.whl (4.8 kB)
   ---------------------------------------- 0.0/545.1 kB ? eta -:--:--
   ---------------------------------------- 545.1/545.1 kB 7.7 MB/s eta 0:00:00
Using cached slicer-0.0.8-py3-none-any.whl (15 kB)
   ---------------------------------------- 0.0/2.8 MB ? eta -:--:--
   ---------------------------------------- 2.8/2.8 MB 30.1 MB/s eta 0:00:00
   ---------------------------------------- 0.0/30.3 MB ? eta -:--:--
   ------- -------------------------------- 6.0/30.3 MB 

In [6]:
pip install xgboost

  Using cached xgboost-3.0.2-py3-none-win_amd64.whl.metadata (2.1 kB)
Using cached xgboost-3.0.2-py3-none-win_amd64.whl (150.0 MB)
Note: you may need to restart the kernel to use updated packages.
